# FIXED COUPON BOND EXAMPLE TREASURY CUSIP 91282CFX4

This is based on a Bloomberg example

In [6]:
%pip install financepy

Note: you may need to restart the kernel to use updated packages.


In [7]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:
import financepy

In [9]:
from financepy.utils import *
from financepy.products.bonds import *

# Define the Bond

In [10]:
issue_date = Date(30, 11, 2022)
maturity_date = Date(30, 11, 2024)
coupon = 0.045
freq_type = FrequencyTypes.SEMI_ANNUAL
accrual_type = DayCountTypes.ACT_ACT_ICMA
face = ONE_MILLION

In [11]:
bond = Bond(issue_date, maturity_date, coupon, freq_type, accrual_type, face)

In [12]:
clean_price = 100+4/32+1/64  # price was quoted as 100-04+

In [13]:
clean_price

100.140625

You can get information about the bond using the print method.

In [14]:
print(bond)

OBJECT TYPE: Bond
ISSUE DATE: 30-NOV-2022
MATURITY DATE: 30-NOV-2024
COUPON (%): 4.5
FREQUENCY: FrequencyTypes.SEMI_ANNUAL
ACCRUAL TYPE: DayCountTypes.ACT_ACT_ICMA
FACE AMOUNT: 1000000


## Bond Cash Flows

We first need to set the settlement date of the bond. 

In [39]:
settlement_date = Date(6, 2, 2023)

print(bond.coupon_dates(settlement_date))

 31-MAY-2023     22500.00 
 30-NOV-2023     22500.00 
 31-MAY-2024     22500.00 
 30-NOV-2024   1022500.00 



In [40]:
# Accrued Interest
ai = bond.calc_accrued_interest(settlement_date)
print(ai)

8406.593406593407


The convention is to use these dates for yield calculations even if some fall on weekends.

## Bond Yield Measures

Current yield is the coupon over the price

In [45]:
print("Current Yield %12.6f %%" % (bond.current_yield(clean_price)*100))

Current Yield     4.493681 %


In [46]:
calcTypes = [YTMCalcType.UK_DMO, YTMCalcType.US_STREET, YTMCalcType.US_TREASURY]

Yield to maturity using different conventions

In [47]:
for ytmCalcType in calcTypes:
    y = bond.yield_to_maturity(settlement_date, clean_price, ytmCalcType)
    print("%30s %12.6f" % (ytmCalcType, y*100))

            YTMCalcType.UK_DMO     4.415114
         YTMCalcType.US_STREET     4.415114
       YTMCalcType.US_TREASURY     4.411863


In [62]:
yieldConvention = YTMCalcType.US_STREET

In [63]:
ytm = bond.yield_to_maturity(settlement_date, clean_price, yieldConvention)

In [64]:
ytm*100

4.415113932558936

Full price is the clean price plus accrued interest

In [65]:
print("Full Price = %12.7f" % bond.full_price_from_ytm(settlement_date, ytm, yieldConvention))

Full Price =  100.9812843


In [66]:
print("Clean Price = %12.7f" % bond.clean_price_from_ytm(settlement_date, ytm, yieldConvention))

Clean Price =  100.1406250


Accrued interest is accrued from previous coupon date to settlement date

In [67]:
print("Previous coupon date is ", bond._pcd)

Previous coupon date is  30-NOV-2022


In [68]:
print("Settlement date is ", settlement_date)

Settlement date is  06-FEB-2023


The amount of accrued interest is 

In [69]:
print("Accrued = %12.2f" % bond._accrued_interest)

Accrued =      8406.59


This is based on the following number of days of accrual

In [70]:
print("Accrued Days = %d" % bond._accrued_days)

Accrued Days = 68


In [71]:
print("Principal = %12.2f" % bond.principal(settlement_date, ytm, yieldConvention))

Principal =   1001406.25


## Bond Risk Measures

The bond dollar duration is the actual derivative with respect to the yield. It is the bond price change for a 1bp drop in the yield-to-maturity divided by 1bp.

In [72]:
duration = bond.dollar_duration(settlement_date, ytm, yieldConvention)
print("Dollar Duration = ", duration)

Dollar Duration =  172.72204606037178


Modified Duration divides the dollar duration by the full price of the bond

In [73]:
modified_duration = bond.modified_duration(settlement_date, ytm, yieldConvention)
print("Modified Duration = ", modified_duration)

Modified Duration =  1.7104362178410772


Macauley Duration multiplies the dollar duration by (1+y/f) and divides by the full price

In [74]:
macauley_duration = bond.macauley_duration(settlement_date, ytm, yieldConvention)
print("Macauley Duration = ", macauley_duration)

Macauley Duration =  1.748195071721795


Convexity is the second derivative of the bond price with respect to the yield-to-maturity

In [75]:
conv = bond.convexity_from_ytm(settlement_date, ytm, yieldConvention)
print("Convexity = ", conv)

Convexity =  0.0383125416304074


Copyright (c) 2022 Dominic O'Kane